In [ ]:
import numpy as np

In [ ]:
def GF_table(bc_left: str, bc_right: str, L, truncation):
    # returns the eigenvalue beta_m and the norm Nx

    m = np.arange(1, truncation+1)
    if (bc_left == 'D' and bc_right == 'D'): #11
        beta_m = m*np.pi/L
        # X_m = np.sin(beta_m*x)
        Nx = L/2
        return beta_m, Nx
    
    if (bc_left == 'D' and bc_right == 'N'): #12
        beta_m = (2*m-1)*np.pi/(2*L)
        # X_m = np.sin(beta_m*x)
        Nx = L/2
        return beta_m, Nx
    
    if (bc_left == 'N' and bc_right == 'D'): #21
        beta_m = (2*m-1)*np.pi/(2*L)
        # X_m = np.sin(beta_m*x)
        Nx = L/2
        return beta_m, Nx
    

def GF_function(x, y, x_s, y_s, Lx, Ly, bc_e: str, bc_n: str, bc_w: str, bc_s: str, truncation = 50):
    # x = np.arange(0, Lx, step)
    # y = np.arange(0, Ly, step)
    beta_m, Nx = GF_table(bc_e, bc_w, Lx, truncation)
    theta_n, Ny = GF_table(bc_s, bc_n, Ly, truncation)
    summation_array = np.zeros(truncation*truncation)
    
    for m in range(1, truncation+1):
        i = m-1

        X_m = np.sin(x*beta_m[i])
        X_m_s = np.sin(x_s*beta_m[i])
        # print("X_m_s: " + str(X_m_s))
        for n in range(1, truncation+1):
            j= n-1

            Y_n = np.sin(y*theta_n[j])
            Y_n_s = np.sin(y_s*theta_n[j])
            # print("X_m_s: " + str(Y_n_s))

            fraction = 1/((beta_m[i]*beta_m[i]) + (theta_n[j]*theta_n[j]))
            X_mult = (X_m*X_m_s)/Nx
            Y_mult = (Y_n*Y_n_s)/Ny

            summation_array[i*truncation + j] = fraction * X_mult * Y_mult
    
    sum = np.sum(summation_array)
    return sum

def GF_1D_function(x, x_s, Length, bc_in: str, bc_out: str, truncation = 50):
    # x = np.arange(0, Lx, step)
    # y = np.arange(0, Ly, step)
    beta_m, Nx = GF_table(bc_in, bc_out, Length, truncation)
    
    summation_array = np.zeros(truncation)
    
    for m in range(1, truncation+1):
        i = m-1

        X_m = np.sin(x*beta_m[i])
        X_m_s = np.sin(x_s*beta_m[i])
        fraction = 1/((beta_m[i]*beta_m[i]))
        X_mult = (X_m*X_m_s)/Nx

        summation_array[i*truncation] = fraction * X_mult
    
    sum = np.sum(summation_array)
    return sum


def return_GF_matrix(x, y, x_s_start, x_s_end, y_s_start, y_s_end, Lx, Ly, bc_e: str, bc_n: str, bc_w: str, bc_s: str, n_step_s = 21):
    # returns GF as a function of x_s and y_s and for a specific x and y value
    step_s_x = (x_s_end-x_s_start)/n_step_s
    step_s_y = (y_s_end-y_s_start)/n_step_s
    x_s = np.linspace(x_s_start, x_s_end, n_step_s)
    # x_s[-1] = x_s_end # just to make sure we arrive to the border
    y_s = np.linspace(y_s_start, y_s_end, n_step_s)
    # y_s[-1] = y_s_end # just to make sure we arrive to the borders

    GF_matrix = np.zeros((n_step_s, n_step_s))

    for i in range(n_step_s):
        for j in range(n_step_s):
            GF_matrix[i, j] = GF_function(x, y, x_s[i], y_s[j], Lx, Ly, bc_e=bc_e, bc_n=bc_n, bc_w=bc_w, bc_s=bc_s)

    return GF_matrix

def return_GF_1D_array(x, x_s_start, x_s_end, Lx, bc_in: str, bc_out: str, n_step_s = 21):
    step_s_x = (x_s_end-x_s_start)/n_step_s

    x_s = np.linspace(0, Lx, n_step_s)
    # x_s[-1] = x_s_end # just to make sure we arrive to the border

    GF_matrix = np.zeros(n_step_s)

    for i in range(n_step_s):
            GF_matrix[i] = GF_1D_function(x, x_s[i], Lx, bc_in = bc_in, bc_out = bc_out)

    return GF_matrix

In [ ]:
Lx = Ly = 1
x_s = y_s = Lx/2
n_step = 2
step = 0.01 # limit might be after 0.01
num_points = 100
# np.linspace(0, L/2, n_step, endpoint=True, retstep=False, dtype=None, axis=0)
# x_in = np.arange(0, L/2, step)
# x_out = np.arange(0, L/2, step)
# y_in = np.arange(0, L/2, step)
# y_out = np.arange(0, L/2, step)
x = np.linspace(0, Lx, num_points)
y = np.linspace(0, Ly, num_points)

GF_function(x[0], y[0], x_s, y_s, Lx, Ly, bc_e = "D", bc_n = "D", bc_w = "D", bc_s = "D")
# green_function(x, y, x_s, y_s, L, bc_e: str, bc_n: str, bc_w: str, bc_s: str, truncation = 10):

In [ ]:
steps = 21
plot_GF = np.zeros((steps, steps))
x = np.linspace(0, Lx, steps)
y = np.linspace(0, Ly, steps)
for i in range(steps):
    for j in range(steps):
        plot_GF[i, j] = GF_function(x[i], y[j], x_s, y_s, Lx, Ly, bc_e = "N", bc_n = "N", bc_w = "D", bc_s = "D")

In [ ]:
import matplotlib.pyplot as plt

# Plot the heatmap
plt.imshow(plot_GF, cmap='jet', interpolation='nearest', origin = 'lower')
plt.colorbar()  # Add a color bar to show the color scale
plt.title('Heatmap of 2D NumPy Array')
plt.xlabel('Column Index')
plt.ylabel('Row Index')
plt.show()

In [ ]:
return_GF_matrix(0.2, 0.2, 0, 1, 0, 1, 1, 1, "D", "D", "D", "D")

In [ ]:
omega = 5
lambda_v = 1
Lx = Ly = 1

# current observer point
x = 0.1
y = 0.1

# source surface area
x_s_start = 0
x_s_end = Lx
y_s_start = 0
y_s_end = Ly

# x_s and y_s array for integral calculation
n_step_s = 21
step_s_x = (x_s_end-x_s_start)/n_step_s
step_s_y = (y_s_end-y_s_start)/n_step_s
x_s = np.linspace(x_s_start, x_s_end, n_step_s)
# x_s[-1] = x_s_end # just to make sure we arrive to the border
y_s = np.linspace(y_s_start, y_s_end, n_step_s)
# y_s[-1] = y_s_end

#boundaries
bc_E = "D"
bc_N = "D"
bc_W = "D"
bc_S = "D"

# boundary values (T or q)
T = 10
q = 3

# # 2D GF integral over surface
# GF_S = return_GF_matrix(x, y, x_s_start, x_s_end, y_s_start, y_s_end, bc_E, bc_N, bc_W, bc_S) # 2D GF(x_s, y_s)
# integral_GF_ys = np.trapz(GF_S*omega, y_s, axis=0) #over y
# integral_GF_ys_xs = np.trapz(integral_GF_ys, x_s) #over x

# # 1D GF as a function of xs and ys
# GF_x = return_GF_1D_array(x, Lx, bc_E, bc_W) # 1D GF(x_s)
# GF_y = return_GF_1D_array(y, Ly, bc_S, bc_N) # 1D GF(y_s)

# # East integral -> Dirichlet
# dGdx = np.diff(GF_S) / np.diff(x_s)


In [ ]:
GF_S = return_GF_matrix(x, y, 0, Lx, 0, Ly, Lx, Ly, bc_E, bc_N, bc_W, bc_S) # 2D GF(x_s, y_s)
# Plot the heatmap
plt.imshow(GF_S, cmap='jet', interpolation='nearest', origin = 'lower')
plt.colorbar()  # Add a color bar to show the color scale
plt.title('Heatmap of 2D NumPy Array')
plt.xlabel('Column Index')
plt.ylabel('Row Index')
plt.show()
# (x, y, x_s_start, x_s_end, y_s_start, y_s_end, Lx, Ly, bc_e: str, bc_n: str, bc_w: str, bc_s: str, n_step_s = 20)

In [ ]:
return_GF_1D_array(x, 0, Lx, Lx, "D", "D")